In [15]:
#automated Model Training
#Importing Libraries
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

df = pd.read_excel("/content/PhonePe_Sherloc_Categories.xlsx")  # Replace with the path to your dataset
df.columns
df2 = df[['payer_name','payer_vpa','payee_account_type','payee_name','payee_vpa','payer_account_type', 'Category1','Category2']]

def custom_tokenizer(text):
    # split the text and value using regular expression
    import re
    pattern = re.compile(r'[a-zA-Z]+\d+')
    text_and_value = pattern.findall(text)
    return text_and_value
# Apply TF-Vectorization on data
tfidf_payer_name = TfidfVectorizer()
tfidf_matrix_payer_name = tfidf_payer_name.fit_transform(df2['payer_name'].astype(str))

tfidf_payee_name = TfidfVectorizer()
tfidf_matrix_payee_name = tfidf_payee_name.fit_transform(df2['payee_name'].astype(str))

tfidf_payee_account_type = TfidfVectorizer()
tfidf_matrix_payee_account_type = tfidf_payee_account_type.fit_transform(df2['payee_account_type'].astype(str))

tfidf_payer_account_type = TfidfVectorizer()
tfidf_matrix_payer_account_type = tfidf_payer_account_type.fit_transform(df2['payer_account_type'].astype(str))

tfidf_payer_vpa = TfidfVectorizer(tokenizer=custom_tokenizer)
df2['payer_vpa'] = df2['payer_vpa'].astype(str)
tfidf_matrix_payer_vpa = tfidf_payer_vpa.fit_transform(df2['payer_vpa'])

tfidf_payee_vpa = TfidfVectorizer(tokenizer=custom_tokenizer)
df2['payee_vpa'] = df2['payee_vpa'].astype(str)
tfidf_matrix_payee_vpa = tfidf_payee_vpa.fit_transform(df2['payee_vpa'])


tfidf_matrix = pd.concat([pd.DataFrame(tfidf_matrix_payer_name.toarray()),
                          pd.DataFrame(tfidf_matrix_payee_name.toarray()),
                          pd.DataFrame(tfidf_matrix_payee_account_type.toarray()),
                          pd.DataFrame(tfidf_matrix_payer_account_type.toarray()),
                          pd.DataFrame(tfidf_matrix_payer_vpa.toarray()),
                          pd.DataFrame(tfidf_matrix_payee_vpa.toarray())], axis=1)
X_train, X_test, y_cat1_train, y_cat1_test, y_cat2_train, y_cat2_test = train_test_split(tfidf_matrix, df2['Category1'], df2['Category2'], test_size=0.2, random_state=42)
clf_cat1 = RandomForestClassifier()

clf_cat1.fit(X_train, y_cat1_train)

clf_cat2 = RandomForestClassifier()
clf_cat2.fit(X_train, y_cat2_train) # Make predictions for each target variable

predictions_cat1 = clf_cat1.predict(X_test)
predictions_cat2 = clf_cat2.predict(X_test)

accuracy_cat1 = accuracy_score(y_cat1_test, predictions_cat1)
print(f"Accuracy for Category Level 1: {accuracy_cat1:.2f}")

# Calculate accuracy for Category Level 2 predictions
accuracy_cat2 = accuracy_score(y_cat2_test, predictions_cat2)
print(f"Accuracy for Category Level 2: {accuracy_cat2:.2f}")

df2.head(2)

# After training the Random Forest classifiers:
with open("clf_cat1.pkl", "wb") as f:
    pickle.dump(clf_cat1, f)

with open("clf_cat2.pkl", "wb") as f:
    pickle.dump(clf_cat2, f)

# After fitting the Tfidf vectorizers:
with open("tfidf_payer_name.pkl", "wb") as f:
    pickle.dump(tfidf_payer_name, f)

with open("tfidf_payee_name.pkl", "wb") as f:
    pickle.dump(tfidf_payee_name, f)

with open("tfidf_payee_account_type.pkl", "wb") as f:
    pickle.dump(tfidf_payee_account_type, f)

with open("tfidf_payer_account_type.pkl", "wb") as f:
    pickle.dump(tfidf_payer_account_type, f)

with open("tfidf_payer_vpa.pkl", "wb") as f:
    pickle.dump(tfidf_payer_vpa, f)

with open("tfidf_payee_vpa.pkl", "wb") as f:
    pickle.dump(tfidf_payee_vpa, f)


<ipython-input-15-463ae51fdedc>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['payer_vpa'] = df2['payer_vpa'].astype(str)
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
<ipython-input-15-463ae51fdedc>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['payee_vpa'] = df2['payee_vpa'].astype(str)


Accuracy for Category Level 1: 0.98
Accuracy for Category Level 2: 0.97


In [16]:
# Pickle file prediction
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
import re

def custom_tokenizer(text):
    pattern = re.compile(r'[a-zA-Z]+\d+')
    return pattern.findall(text)

def predict_categories(input_data):
    # Load the saved models and vectorizers
    clf_cat1 = pickle.load(open("clf_cat1.pkl", "rb"))
    clf_cat2 = pickle.load(open("clf_cat2.pkl", "rb"))

    tfidf_payer_name = pickle.load(open("tfidf_payer_name.pkl", "rb"))
    tfidf_payee_name = pickle.load(open("tfidf_payee_name.pkl", "rb"))
    tfidf_payee_account_type = pickle.load(open("tfidf_payee_account_type.pkl", "rb"))
    tfidf_payer_account_type = pickle.load(open("tfidf_payer_account_type.pkl", "rb"))
    tfidf_payer_vpa = pickle.load(open("tfidf_payer_vpa.pkl", "rb"))
    tfidf_payee_vpa = pickle.load(open("tfidf_payee_vpa.pkl", "rb"))
    # Transform input data using the TFIDF vectorizers
    payer_name = tfidf_payer_name.transform([input_data['payer_name']])
    payee_name = tfidf_payee_name.transform([input_data['payee_name']])
    payee_account_type = tfidf_payee_account_type.transform([input_data['payee_account_type']])
    payer_account_type = tfidf_payer_account_type.transform([input_data['payer_account_type']])
    payer_vpa = tfidf_payer_vpa.transform([input_data['payer_vpa']])
    payee_vpa = tfidf_payee_vpa.transform([input_data['payee_vpa']])

    tfidf_matrix = pd.concat([pd.DataFrame(payer_name.toarray()),
                              pd.DataFrame(payee_name.toarray()),
                              pd.DataFrame(payee_account_type.toarray()),
                              pd.DataFrame(payer_account_type.toarray()),
                              pd.DataFrame(payer_vpa.toarray()),
                              pd.DataFrame(payee_vpa.toarray())], axis=1)

    # Predict
    prediction_cat1 = clf_cat1.predict(tfidf_matrix)
    prediction_cat2 = clf_cat2.predict(tfidf_matrix)

    return prediction_cat1[0], prediction_cat2[0]

# Example usage
input_data = {
    'payer_name': 'John Doe',
    'payer_vpa': 'john.doe@bank',
    'payee_account_type': 'savings',
    'payee_name': 'Jane Smith',
    'payee_vpa': 'jane.smith@bank',
    'payer_account_type': 'current'
}

category1, category2 = predict_categories(input_data)
print(f"Predicted Category Level 1: {category1}")
print(f"Predicted Category Level 2: {category2}")


Predicted Category Level 1: Personal Transfer
Predicted Category Level 2: TRANSFER SELF


In [17]:
# Model Monitoring Pipeline
def print_accuracy():
    test_data = pd.read_excel("/content/PhonePe_Sherloc_Categories.xlsx")

    # Load the saved vectorizers
    with open("tfidf_payer_name.pkl", "rb") as f:
        tfidf_payer_name = pickle.load(f)
    with open("tfidf_payee_name.pkl", "rb") as f:
        tfidf_payee_name = pickle.load(f)
    with open("tfidf_payee_account_type.pkl", "rb") as f:
        tfidf_payee_account_type = pickle.load(f)
    with open("tfidf_payer_account_type.pkl", "rb") as f:
        tfidf_payer_account_type = pickle.load(f)
    with open("tfidf_payer_vpa.pkl", "rb") as f:
        tfidf_payer_vpa = pickle.load(f)
    with open("tfidf_payee_vpa.pkl", "rb") as f:
        tfidf_payee_vpa = pickle.load(f)

    # Transform test data using the saved vectorizers
    payer_name = tfidf_payer_name.transform(test_data['payer_name'].astype(str))
    payee_name = tfidf_payee_name.transform(test_data['payee_name'].astype(str))
    payee_account_type = tfidf_payee_account_type.transform(test_data['payee_account_type'].astype(str))
    payer_account_type = tfidf_payer_account_type.transform(test_data['payer_account_type'].astype(str))
    payer_vpa = tfidf_payer_vpa.transform(test_data['payer_vpa'].astype(str))
    payee_vpa = tfidf_payee_vpa.transform(test_data['payee_vpa'].astype(str))

    tfidf_matrix = pd.concat([pd.DataFrame(payer_name.toarray()),
                              pd.DataFrame(payee_name.toarray()),
                              pd.DataFrame(payee_account_type.toarray()),
                              pd.DataFrame(payer_account_type.toarray()),
                              pd.DataFrame(payer_vpa.toarray()),
                              pd.DataFrame(payee_vpa.toarray())], axis=1)

    # Predict using the loaded models
    with open("clf_cat1.pkl", "rb") as f:
        clf_cat1 = pickle.load(f)
    with open("clf_cat2.pkl", "rb") as f:
        clf_cat2 = pickle.load(f)

    predictions_cat1 = clf_cat1.predict(tfidf_matrix)
    predictions_cat2 = clf_cat2.predict(tfidf_matrix)

    accuracy_cat1 = accuracy_score(test_data['Category1'], predictions_cat1)
    accuracy_cat2 = accuracy_score(test_data['Category2'], predictions_cat2)

    print(f"Accuracy for Category Level 1: {accuracy_cat1:.2f}")
    print(f"Accuracy for Category Level 2: {accuracy_cat2:.2f}")
# Example usage
input_data = {
    'payer_name': 'John Doe',
    'payer_vpa': 'john.doe@bank',
    'payee_account_type': 'savings',
    'payee_name': 'Jane Smith',
    'payee_vpa': 'jane.smith@bank',
    'payer_account_type': 'current'
}

category1, category2 = predict_categories(input_data)
print(f"Predicted Category Level 1: {category1}")
print(f"Predicted Category Level 2: {category2}")

# Print accuracy using the stored test data
print_accuracy()


Predicted Category Level 1: Personal Transfer
Predicted Category Level 2: TRANSFER SELF
Accuracy for Category Level 1: 0.98
Accuracy for Category Level 2: 0.98
